# 20 Sklearn Features You Didn't Know About | P(Guarantee) = 0.75
## Step up your Sklearn game by learning new cool stuff
![](images/unsplash.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@alevisionco?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>alevision.co</a>
        on 
        <a href='https://unsplash.com/s/photos/secret?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>